In [ ]:
!pip install transformers
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
from transformers import AlignProcessor, AlignModel
from torch import Tensor
import torch
from PIL import Image
import pandas as pd
import os
import numpy as np
import csv
import pandas as pd
import json
import cv2


In [ ]:
drive.mount('/content/drive')
FOLDER_PATH = "/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings/Images"
EMB_PATHS="/content/drive/MyDrive/TFM/Embeddings/ImgEmbeddings/ALIGNEmbeddings.csv"
TEXT_PATH = "/content/drive/MyDrive/TFM/Embeddings/Posts.json"


Mounted at /content/drive


In [ ]:
captions = {}
with open(TEXT_PATH, "r") as input:
  for line in input:
    post = json.loads(line)
    captions[post["_id"]] = post["caption"]

In [ ]:
processor = AlignProcessor.from_pretrained("kakaobrain/align-base")
model = AlignModel.from_pretrained("kakaobrain/align-base")

In [ ]:
s

In [ ]:
with open(EMB_PATHS, 'a') as file:
  writer = csv.writer(file)
  for folder_name in os.listdir(FOLDER_PATH):

    folder_path = os.path.join(FOLDER_PATH, folder_name)

    if os.path.isdir(folder_path):
      image_arrays = []
      emb_array=[]
      img_ids = [folder_name]

      capt = captions[folder_name]
      if capt is None:
        capt = "w"
      if len(capt) > 512:
        capt = capt[:512]

      for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(image_path)

            image_arrays.append(img)
        if os.path.isfile(image_path) and image_name.lower().endswith(('.mp4')):
          cap = cv2.VideoCapture(image_path)

          ret, frame = cap.read()
          image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
          cap.release()
          image_arrays.append(img)
      inputs = processor(text=capt, images=image_arrays, return_tensors="pt")
      with torch.no_grad():
        outputs = model(**inputs)

        emb_array = outputs.text_embeds.numpy().flatten().tolist() + outputs.text_embeds.numpy().flatten().tolist()
        #print(len(emb_array))

      writer.writerow(emb_array)

